In [23]:
import numpy as np
import cv2
from keras.models import load_model
from keras.preprocessing.image import img_to_array
from keras.applications.mobilenet_v2 import preprocess_input

In [24]:
cap=cv2.VideoCapture(0)

In [25]:
def detect_face(frame,faceNet,model):
    h,w=frame.shape[:2]
    blob=cv2.dnn.blobFromImage(frame,1,(224,224),(104,116,124))
    faceNet.setInput(blob)
    detections=faceNet.forward()
    faces=[]
    locs=[]
    preds=[]
    for i in range(0,detections.shape[2]):
        confidence=detections[0,0,i,2]
        if confidence > 0.5:
            box=detections[0,0,i,3:7]*np.array([w,h,w,h])
            (startx,starty,endx,endy)=box.astype('int')
            (startx,statry)=(max(0,startx),max(0,starty))
            (endx,endy)=(min(w-1,endx),min(h-1,endy))
            face=frame[starty:endy,startx:endx]
            face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
            face=cv2.resize(face,(224,224))
            face=img_to_array(face)
            face=preprocess_input(face)
            faces.append(face)
            faces.append(face)
            locs.append((startx,starty,endx,endy))
    if len(faces)>0:
        faces=np.array(faces,dtype='float32')
        preds=model.predict(faces,batch_size=32)
    return (locs,preds)


In [ ]:
model=load_model('face_mask.h5')
faceNet=cv2.dnn.readNet('deploy.prototxt','res10_300x300_ssd_iter_140000.caffemodel')
while True:
    ret,frame=cap.read()
    (locs,preds)=detect_face(frame,faceNet,model)
    for box,pred in zip(locs,preds):
        (startx,starty,endx,endy)=box
        mask,withoutmask=pred
        label='Mask' if mask > withoutmask else 'no mask'
        color =(0,255,0) if label == 'Mask' else (0,0,255)
        cv2.putText(frame,label,(startx,starty-10),cv2.FONT_HERSHEY_SIMPLEX,.45,color,2)
        cv2.rectangle(frame,(startx,starty),(endx,endy),color,2)
    cv2.imshow('Webcam ',frame)
    if cv2.waitKey(1)==13:
        break
cap.release()
cv2.destroyAllWindows()